**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [10]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [11]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [12]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-20 12:36:27.579 | SUCCESS  | new_helper:load_variable:317 - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-20 12:36:27.581 | INFO     | __main__:<module>:3 - b'ea82bfd4d4410e5bf81677f20af87196c90af7e1b8616d573d38359207a30422'


2 - Creates a user account into the blockchain.

In [13]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-20 12:36:27.630 | INFO     | __main__:<module>:13 - User Role: author
2025-02-20 12:36:27.632 | INFO     | __main__:<module>:24 - Creating account with name: jolly_noether
2025-02-20 12:36:27.634 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-20 12:36:27.655 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'ea82bfd4d4410e5bf81677f20af87196c90af7e1b8616d573d38359207a30422'
2025-02-20 12:36:27.658 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: EAA641C8CF286241F52BCD3BFA4407D309A14CEF
2025-02-20 12:36:27.660 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-20 12:36:27.661 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-02-20 12:36:27,674 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-20 12:36:28,713 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025

3 - Queries the blockchain to confirm the succesful creation of the account.

In [14]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-20 12:36:28.746 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-20 12:36:28,748 - WARNING - create_account_contract_address: EAA641C8CF286241F52BCD3BFA4407D309A14CEF
2025-02-20 12:36:28,762 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-20 12:36:28,764 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-20 12:36:31,748 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-20 12:36:31,758 - INFO - ('COMMITTED', 5, 0)
2025-02-20 12:36:31,759 - WARNING - Account Data: {'address': 'EAA641C8CF286241F52BCD3BFA4407D309A14CEF', 'hex_hash': 'f3532a7156ca3c7f657a360ad542f3d65354c1244a9ff6782168246a908e878f', 'timestamp': 1740054991759}
2025-02-20 12:36:31.760 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-20 12:36:31,764 - INFO - Appended data under 'jolly_noether@test' in logs/account_data.json
2025-02-20 12:36:31.767 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-20 12:3

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [15]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-20 12:36:31.800 | SUCCESS  | new_helper:load_variable:317 - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-20 12:36:31.802 | INFO     | __main__:<module>:3 - b'3f558ff740a962eccf6fcd3d79903cb5328ce832beb888f2d4bdffeea3bdc7bc'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [16]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-20 12:36:31.813 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-20 12:36:31.827 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'3f558ff740a962eccf6fcd3d79903cb5328ce832beb888f2d4bdffeea3bdc7bc'
2025-02-20 12:36:31.829 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 03E61119F8C4FFC56025E7431C9F210784A60A63
2025-02-20 12:36:31.831 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-20 12:36:31.832 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-20 12:36:31,834 - INFO - Processing account: jolly_noether@test
2025-02-20 12:36:31,836 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Jolly Noether', 'foaf:mbox': 'jolly_noether@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Morris College'}, 'schema:identifier': {'

6 - Queries the blockchain again to confirm the succesfull details setting.

In [17]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details)

2025-02-20 12:36:34.861 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-20 12:36:34,863 - WARNING - create_account_contract_address: EAA641C8CF286241F52BCD3BFA4407D309A14CEF
2025-02-20 12:36:34,877 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-20 12:36:34,879 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-20 12:36:37,837 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-20 12:36:37,847 - INFO - ('COMMITTED', 5, 0)
2025-02-20 12:36:37,851 - WARNING - Account Data: {'address': 'EAA641C8CF286241F52BCD3BFA4407D309A14CEF', 'hex_hash': '7f59fc1eebaed979089816061245eccef9aea82ad5cc28297de10f72cfcbe023', 'timestamp': 1740054997850}
2025-02-20 12:36:37.852 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-20 12:36:37,856 - INFO - Appended data under 'jolly_noether@test' in logs/account_data.json
2025-02-20 12:36:37.857 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-20 12:3

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [18]:
user_metadata = download_json_from_ipfs(account_metadata_cid)


2025-02-20 12:36:37.917 | SUCCESS  | ipfs_functions:download_json_from_ipfs:52 - Success downloading JSON from IPFS. Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'jolly_noether@test', 'schema:publicKey': '93b24e83e5a01970f786207eb4d3d194731fed63daf76ed9ee31c1ac83310ab6', 'schema:roleName': 'author'}, 'foaf:mbox': 'jolly_noether@email.com', 'foaf:name': 'Jolly Noether', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Morris College'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '9833-6461-2701-X'}}
